In [7]:
import os
import openai
from IPython.display import display, Markdown, Latex, HTML, JSON

MODEL_NAME = 'gpt-3.5-turbo'

with open('/home/loc/Documents/OPENAI_API_KEY.txt') as f:
    OPENAI_API_KEY = f.read().strip()
    os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY
    openai.api_key = OPENAI_API_KEY

def get_completion(prompt, model=MODEL_NAME, temperature=0):
    # generate message
    messages = [{'role':'user',
                 'content':prompt}
               ]
    # receive the response
    response = openai.ChatCompletion.create(
        model = model,
        messages = messages,
        temperature = temperature # this is the degree of randomness of model
    )
    return response.choices[0].message['content']


def get_completion_from_messages(messages,model=MODEL_NAME, temperature=0):
    response = openai.ChatCompletion.create(
        model = model,
        messages = messages,
        temperature = temperature # this is the degree of randomness of model
    )
    return response.choices[0].message['content']

### context conversation
```conversation = roles + messages```

**roles**
- `system`: Sets behavior of assistant
- `assistant`: Chat model
- `user` You

In [8]:
messages =  [  
{'role':'system', 'content':'You are an assistant that speaks like Shakespeare.'},    
{'role':'user', 'content':'tell me a joke'},   
{'role':'assistant', 'content':'Why did the chicken cross the road'},   
{'role':'user', 'content':'I don\'t know'}  ]

In [9]:
response = get_completion_from_messages(messages, temperature=1)
print(response)

To get to the other side, of course! Alas, a classic jest that tickles the funny bone.


### help your model focus on the conversation

In [10]:
messages =  [  
{'role':'system', 'content':'You are friendly chatbot.'},    
{'role':'user', 'content':'Hi, my name is Isa'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

Hello Isa! It's nice to meet you. How can I assist you today?


In [11]:
messages =  [  
{'role':'system', 'content':'You are friendly chatbot.'},    
{'role':'user', 'content':'Yes,  can you remind me, What is my name?'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

I'm sorry, but I don't have access to personal information about you, including your name. However, please feel free to share your name with me if you'd like!


In [12]:
messages =  [  
{'role':'system', 'content':'You are friendly chatbot.'},
{'role':'user', 'content':'Hi, my name is Isa'},
{'role':'assistant', 'content': "Hi Isa! It's nice to meet you. \
Is there anything I can help you with today?"},
{'role':'user', 'content':'Yes, you can remind me, What is my name?'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

Your name is Isa. Let me know if you need any more reminders or assistance!


### OrderBot
We can automate the collection of user prompts and assistant responses to build a  OrderBot. The OrderBot will take orders at a pizza restaurant. 

In [23]:
context = [ {'role':'system', 'content':"""
You are OrderBot, an automated service to collect orders for a pizza restaurant. \
You first greet the customer, then collects the order, \
and then asks if it's a pickup or delivery. \
You wait to collect the entire order, then summarize it and check for a final \
time if the customer wants to add anything else. \
If it's a delivery, you ask for an address. \
Finally you collect the payment.\
Make sure to clarify all options, extras and sizes to uniquely \
identify the item from the menu.\
You respond in a short, very conversational friendly style. \
The menu includes \
pepperoni pizza  12.95, 10.00, 7.00 \
cheese pizza   10.95, 9.25, 6.50 \
eggplant pizza   11.95, 9.75, 6.75 \
fries 4.50, 3.50 \
greek salad 7.25 \
Toppings: \
extra cheese 2.00, \
mushrooms 1.50 \
sausage 3.00 \
canadian bacon 3.50 \
AI sauce 1.50 \
peppers 1.00 \
Drinks: \
coke 3.00, 2.00, 1.00 \
sprite 3.00, 2.00, 1.00 \
bottled water 5.00 \
"""} ]  # accumulate messages

current_context = context.copy()

current_context

[{'role': 'system',
  'content': "\nYou are OrderBot, an automated service to collect orders for a pizza restaurant. You first greet the customer, then collects the order, and then asks if it's a pickup or delivery. You wait to collect the entire order, then summarize it and check for a final time if the customer wants to add anything else. If it's a delivery, you ask for an address. Finally you collect the payment.Make sure to clarify all options, extras and sizes to uniquely identify the item from the menu.You respond in a short, very conversational friendly style. The menu includes pepperoni pizza  12.95, 10.00, 7.00 cheese pizza   10.95, 9.25, 6.50 eggplant pizza   11.95, 9.75, 6.75 fries 4.50, 3.50 greek salad 7.25 Toppings: extra cheese 2.00, mushrooms 1.50 sausage 3.00 canadian bacon 3.50 AI sauce 1.50 peppers 1.00 Drinks: coke 3.00, 2.00, 1.00 sprite 3.00, 2.00, 1.00 bottled water 5.00 "}]

In [24]:
def collect_messages(user_message):
    # Append user message
    current_context.append({'role':'user','content':f"{user_message}"})
    # Send API
    response = get_completion_from_messages(current_context)
    # Append model output
    current_context.append({'role':'user','content':f"{user_message}"})
    return response

In [26]:
current_context = context.copy()

while True:
    # Enter your message
    user_message = input()
    # Get response and collect messages
    print(len(current_context),"\n",collect_messages(user_message))

 Hello


1 
 Hello! Welcome to our pizza restaurant! What can I get for you today?


 Can you suggest me your product?


3 
 Of course! We have pepperoni pizza for $12.95, cheese pizza for $10.95, eggplant pizza for $11.95, fries for $4.50, greek salad for $7.25, and drinks like coke for $3.00. What would you like to order?


 How much for a pepperoni pizza?


...

 No, i think that is enough


17 
 Great! So you'd like a medium pepperoni pizza and 2 cokes. Is that correct?


 Yes


19 
 Great! Here's your order: 

- 1 medium pepperoni pizza for $10.00
- 2 cokes for $6.00

Is there anything else you'd like to add?


KeyboardInterrupt: Interrupted by user

In [29]:
messages =  current_context.copy()
messages.append(
{'role':'system', 'content':'create a json summary of the previous food order. Itemize the price for each item\
 The fields should be 1) pizza, include size and price 2) list of toppings 3) list of drinks, include size and price   4) list of sides include size and price 5)total price '},    
)
 #The fields should be 1) pizza, price 2) list of toppings 3) list of drinks, include size include price  4) list of sides include size include price, 5)total price '},    

response = get_completion_from_messages(messages, temperature=0)
print(response)

{
  "pizza": {
    "type": "pepperoni",
    "size": "medium",
    "price": 12.95
  },
  "toppings": [],
  "drinks": [
    {
      "type": "coke",
      "size": "regular",
      "price": 3.00
    },
    {
      "type": "coke",
      "size": "regular",
      "price": 3.00
    }
  ],
  "sides": [],
  "total price": 18.95
}
